In [16]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import LinearSVC

In [17]:
reviews = pd.read_csv('../input/train.csv', sep = ',')

In [18]:
reviews = reviews.drop(["Product Name", "Brand Name", "Price"], axis=1)
reviews.head()

,Id,Reviews,Rating
0,1,I feel so LUCKY to have found this used (phone...,5
1,2,"nice phone, nice up grade from my pantach revu...",4
2,3,Very pleased,5
3,4,It works good but it goes slow sometimes but i...,4
4,5,Great phone to replace my lost phone. The only...,4


In [19]:
reviews.head()

,Id,Reviews,Rating
0,1,I feel so LUCKY to have found this used (phone...,5
1,2,"nice phone, nice up grade from my pantach revu...",4
2,3,Very pleased,5
3,4,It works good but it goes slow sometimes but i...,4
4,5,Great phone to replace my lost phone. The only...,4


In [20]:
print(reviews['Reviews'].isnull().sum())
reviews['Reviews'] = reviews['Reviews'].fillna('')
print(reviews['Reviews'].isnull().sum())

26
0


In [21]:
print(reviews['Rating'].isnull().sum())

0


In [22]:
reviews.dtypes
reviews['Reviews'] = reviews.Reviews.apply(str)
# reviews['Reviews'] = reviews['Reviews'].astype(str)
reviews.dtypes

Id          int64
Reviews    object
Rating      int64
dtype: object

In [23]:
reviews['Reviews'] = [review.lower() for review in reviews['Reviews']]

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
# This vectorizer breaks text into single words and bi-grams
# and then calculates the TF-IDF representation
vectorizer = TfidfVectorizer(ngram_range=(1,2))
 
# the 'fit' builds up the vocabulary from all the reviews
# while the 'transform' step turns each indivdual text into
# a matrix of numbers.
reviews['Vectorized'] = vectorizer.fit_transform(reviews.Reviews)

In [25]:
test = pd.read_csv('../input/test.csv', sep = ',')

In [26]:
print(test['Reviews'].isnull().sum())
test['Reviews'] = test['Reviews'].fillna('')
print(test['Reviews'].isnull().sum())

36
0


In [27]:
test['Reviews'] = test.Reviews.apply(str)

In [28]:
test['Reviews'] = [review.lower() for review in test['Reviews']]

In [29]:
test.head()

,Id,Product Name,Brand Name,Price,Reviews
0,202646,Huawei P8 Lite ALE-L23 Factory Unlocked 16GB L...,Huawei,184.9,excelent
1,202647,Huawei P8 Lite ALE-L23 Factory Unlocked 16GB L...,Huawei,184.9,fit perfectly. excelent.
2,202648,Huawei P8 Lite ALE-L23 Factory Unlocked 16GB L...,Huawei,184.9,"it has a very good relation, price/performance."
3,202649,Huawei P8 Lite ALE-L23 Factory Unlocked 16GB L...,Huawei,184.9,like it a lot
4,202650,Huawei P8 Lite ALE-L23 Factory Unlocked 16GB L...,Huawei,184.9,"dear , i had a problem wich the wifi senial !!..."


In [30]:
test = test.drop(["Product Name", "Brand Name", "Price"], axis=1)

In [31]:
test['Vectorized'] = vectorizer.transform(test.Reviews)

In [32]:
classf = LinearSVC()
classf.fit(reviews['Vectorized'][0], reviews['Rating'])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [33]:
test['Rating'] = classf.predict(test['Vectorized'][0])

In [34]:
test = test.drop(["Reviews", "Vectorized"], axis=1)

In [35]:
test.to_csv('submission.csv', index=False)